# Simulate acoustic wave propagation with user-defined material properties

The OptimUS library provides functionality to simulate acoustic wave propagation in unbounded domains with homogeneous scatterers. This notebook demonstrates functionality to load different materials or define new materials.

## Import the OptimUS library

Load the OptimUS library.

In [1]:
# Make sure the OptimUS library is in the Python path.
import sys
sys.path.append('..')

In [2]:
import optimus

## Specify the geometry and source field

Let us consider a plane wave field and a spherical scatterer.

In [3]:
frequency = 1000
source = optimus.source.create_planewave(frequency)
geometry = optimus.geometry.shapes.Sphere(element_size=0.4)

## Specify the materials of the domains

The OptimUS library has a built-in database with most common materials. Alternatively, other materials can be defined by specifying their acoustic properties.

### Load parameters from the material database

A table with material properties stored in the default database can be retrieved from the library as follows.

In [4]:
materials_table = optimus.material.common.get_excel_database(database='default')

The material table is a Pandas dataframe. Let us inspect the first five materials.

In [5]:
materials_table.head(5)

Tissue Density (kg/m3) Speed of Sound (m/s)  \
            Name         Average              Average   
0  Adrenal Gland      928.000000          1500.000000   
1            Air        1.164092           343.000000   
2           Bile      928.000000          1500.000000   
3          Blood     1049.750000          1578.183333   
4   Blood Plasma     1020.050000          1549.350000   

  Nonlinearity Parameter B/A Attenuation Constant          \
                     Average         a [Np/m/MHz]       b   
0                        NaN            15.492000  1.0000   
1                        NaN             0.039144  2.0000   
2                     6.0000             0.255000  1.3100   
3                     6.1125             2.367600  1.0498   
4                     5.7400             1.075000  1.1596   

  Heat Capacity (J/kg/°C) Thermal Conductivity (W/m/°C)  \
                  Average                       Average   
0             3512.500000                      0.443250   
1             1003.666667                      0.027382   
2             4037.000000                      0.583250   
3             3617.000000                      0.516857   
4             3930.000000                      0.582000   

  Heat Transfer Rate (ml/min/kg) Heat Generation Rate (W/kg)  
                         Average                     Average  
0                    1457.755102                   22.576098  
1                       0.000000                    0.000000  
2                       0.000000                    0.000000  
3                   10000.000000                    0.000000  
4                            NaN                         NaN

The complete table can also be displayed.

In [6]:
materials_table.style.set_properties(**{'border': '1.3px solid green','color': 'blue'})

Let us load fat from the standard database and display its properties.

In the case of multiple domains, a list of materials should be passed to the function. Furthermore, the name of the material is case insensitive.

In [7]:
material_fat = optimus.material.load_material('fat')
material_fat.print()

name  density  speed_of_sound  attenuation_coeff_a  attenuation_pow_b
 fat    911.0       1440.1875               4.3578             1.0861


### Define a new material

New materials can be created with the `create_material` function, which needs a name, the density and the speed of sound in the material. If not specified, attenuation is set to zero.

The material parameters are: name (string), density (float), speed of sound (float), attenuation_coeff_a (float), attenuation_pow_b (float). The units are SI, as shown in the table above. 

In [8]:
material_user_1 = optimus.material.create_material(name='new-material-1',
                                                   density=2500,
                                                   speed_of_sound=1500)
material_user_1.print()

           name  density  speed_of_sound  attenuation_coeff_a  attenuation_pow_b
 new-material-1   2500.0          1500.0                  0.0                0.0


Alternatively, material properties available from a dictionary can be passed to the `create_material` function. The keys of the dictionary need to match the attribute names and unpacked before passing to the function.

In [9]:
properties = {
    "name": 'fat',
    "density": 2000,
    "speed_of_sound": 4000,
    "attenuation_coeff_a": 50,
    "attenuation_pow_b": 1.5,
}

In [10]:
material_user_2 = optimus.material.create_material(**properties)

In [11]:
material_user_2.print()

name  density  speed_of_sound  attenuation_coeff_a  attenuation_pow_b
 fat   2000.0          4000.0                 50.0                1.5


### Write material to user defined database

The OptimUS library provides two material databases: a default and user-defined one. The user-defined database can be filled with any new material and later used in other simulations, because the database is stored on disk.

By default, the user-defined database contains one dummy material.

In [12]:
optimus.material.common.get_excel_database(database='user-defined', index_col=0)

Tissue Density (kg/m3) Speed of Sound (m/s)  \
           Name         Average              Average   
0  new-material               0                    0   

  Nonlinearity Parameter B/A Attenuation Constant    Heat Capacity (J/kg/°C)  \
                     Average         a [Np/m/MHz]  b                 Average   
0                          0                    0  0                       0   

  Thermal Conductivity (W/m/°C) Heat Transfer Rate (ml/min/kg)  \
                        Average                        Average   
0                             0                              0   

  Heat Generation Rate (W/kg)  
                      Average  
0                           0

A user-defined material can be written into the database by setting the `save_to_file` argument to `True`.

In [13]:
my_material = optimus.material.create_material(name='my-new-material',
                                               density=2500,
                                               speed_of_sound=1492,
                                               save_to_file=True,
                                               )
my_material.print()

            name  density  speed_of_sound  attenuation_coeff_a  attenuation_pow_b
 my-new-material   2500.0          1492.0                  0.0                0.0


In [14]:
optimus.material.common.get_excel_database(database='user-defined', index_col=0)

Tissue Density (kg/m3) Speed of Sound (m/s)  \
              Name         Average              Average   
0     new-material               0                    0   
1  my-new-material            2500                 1492   

  Nonlinearity Parameter B/A Attenuation Constant    Heat Capacity (J/kg/°C)  \
                     Average         a [Np/m/MHz]  b                 Average   
0                        0.0                    0  0                     0.0   
1                        NaN                    0  0                     NaN   

  Thermal Conductivity (W/m/°C) Heat Transfer Rate (ml/min/kg)  \
                        Average                        Average   
0                           0.0                            0.0   
1                           NaN                            NaN   

  Heat Generation Rate (W/kg)  
                      Average  
0                         0.0  
1                         NaN

The material can be loaded in other simulation as usual.

In [15]:
my_material_again = optimus.material.load_material('my-new-material')
my_material_again.print()

            name  density  speed_of_sound  attenuation_coeff_a  attenuation_pow_b
 my-new-material   2500.0          1492.0                  0.0                0.0


Note that attempting to create a new material with the same name as a preexisting one returns a value error.

In [16]:
my_material = optimus.material.create_material(name='my-new-material',
                                               density=1500,
                                               speed_of_sound=1000,
                                               save_to_file=True,
                                               )

ValueError: A material with the name: [1mmy-new-material[0m  already EXISTs in the database files (either default or user-defined).